In [7]:
from pathlib import Path
from itertools import chain
import pandas as pd
import numpy as np

import json
import sys
sys.path.insert(0, '..')

from predictor.models import LinearModel
from predictor.preprocessing import VALUE_COLS, VEGETATION_LABELS
from predictor.models import nn_FeedForward
from predictor.analysis import plot_shap_values

In [5]:
with open('../data/processed/normalizing_dict.json', 'r') as f:
    normalizing_dict = json.load(f)
    
path_to_arrays = Path('../data/processed/arrays')

In [9]:
model = nn_FeedForward(path_to_arrays, hide_vegetation=True)
model

In [10]:
model.train(num_epochs=5)

Training model without vegetation features


  0%|          | 0/8175 [00:00<?, ?it/s]

After split, training on 261593 examples, validating on 29065 examples


  0%|          | 27/8175 [00:00<00:30, 266.30it/s]

Epoch 0 - Training RMSE: 0.06997232270623566, Validation RMSE: 0.06983803885162074


...

Epoch 2 - Training RMSE: 0.05542360727917346, Validation RMSE: 0.0645962596365852


  0%|          | 26/8175 [00:00<00:31, 256.86it/s]

Epoch 3 - Training RMSE: 0.054586497671013576, Validation RMSE: 0.06387223563145752


100%|██████████| 909/909 [00:01<00:00, 803.46it/s]

Epoch 4 - Training RMSE: 0.054150071823314425, Validation RMSE: 0.06281156092472333


In [11]:
model.evaluate()

100%|██████████| 916/916 [00:01<00:00, 619.96it/s]


Test set RMSE: 0.08204460144042969


In [12]:
background_data = model.load_tensors(mode='train')
test_data = model.load_tensors(mode='test')

Training model without vegetation features
